<a href="https://colab.research.google.com/github/KayvanShah1/usc-dsci553-data-mining-sp24/blob/main/assignment-5/notebooks/HW5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation & Setup

In [1]:
%pip install pyspark ipython-autotime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=c527dd7cc52d3154f7065f26a608359f1865b46fc67d3f458596103573421ee5
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
%%bash
java --version
pyspark --version

openjdk 11.0.22 2024-01-16
OpenJDK Runtime Environment (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1)
OpenJDK 64-Bit Server VM (build 11.0.22+7-post-Ubuntu-0ubuntu222.04.1, mixed mode, sharing)


Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.5.1
      /_/
                        
Using Scala version 2.12.18, OpenJDK 64-Bit Server VM, 11.0.22
Branch HEAD
Compiled by user heartsavior on 2024-02-15T11:24:58Z
Revision fd86f85e181fc2dc0f50a096855acf83a6cc5d9c
Url https://github.com/apache/spark
Type --help for more information.


# Imports

In [3]:
import os
import sys
import json
import pandas as pd
from itertools import combinations
import math
import statistics
from pyspark import SparkContext
import numpy as np
import warnings

from pprint import pprint

warnings.simplefilter("ignore")

%load_ext autotime

time: 431 µs (started: 2024-04-12 23:50:40 +00:00)


# Configuration

In [4]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/DSCI553/hw5")


class Path:
    current_dir = os.getcwd()
    data_dir = os.path.join(current_dir, "data")
    input_csv_file = os.path.join(data_dir, "users.txt")

    output_dir = os.path.join(current_dir, "output")
    os.makedirs(output_dir, exist_ok=True)

    task1_output = os.path.join(output_dir, "task1_op.txt")
    task2_output = os.path.join(output_dir, "task2_op.txt")
    task3_output = os.path.join(output_dir, "task3_op.txt")

time: 716 ms (started: 2024-04-12 23:50:40 +00:00)


# Task 1

In [5]:
import binascii
import csv
import random
import sys
import time

from blackbox import BlackBox
from pyspark import SparkConf, SparkContext

FILTER_ARRAY_LENGTH = 69997
ENCODING = "utf-8"
NUM_HASHES = 50
PRIME_NUMBER = 1e9 + 7


def generate_hash_function_params(max_range, count):
    """Generate random hash function parameters within a specified range."""
    a = random.sample(range(1, max_range), count)  # Random coefficient 'a'
    b = random.sample(range(1, max_range), count)  # Random intercept 'b'
    return list(zip(a, b))


def hash_user(user, params):
    """Hash an item using given hash function parameters.
    Calculate hash value using the formula: ((a * item + b) % PRIME_NUMBER) % num_bins
    """
    user = int(binascii.hexlify(user.encode("utf8")), 16)
    hash_val = ((params[0] * user + params[1]) % PRIME_NUMBER) % FILTER_ARRAY_LENGTH
    return hash_val


def myhashs(user):
    hash_funcs = generate_hash_function_params(FILTER_ARRAY_LENGTH, NUM_HASHES)
    return [hash_user(user, hash_funcs[i]) for i in range(NUM_HASHES)]


def bloom_filter(input_path: str, blackbox: BlackBox, num_of_asks: int, stream_size: int):
    results = []
    exist_user = set()
    exist_hash = []

    # Fetch stream and perform Bloom Filtering for each batch
    for i in range(num_of_asks):
        stream_users = blackbox.ask(input_path, stream_size)
        false_positives = 0
        for user in stream_users:
            usr_hashes = myhashs(user)

            if usr_hashes in exist_hash and user not in exist_user:
                false_positives += 1

            exist_hash.append(usr_hashes)
            exist_user.add(user)

        results.append([i, false_positives / stream_size])

    return results


def save_output(output_file_name, results):
    header = ["Time", "FPR"]
    with open(output_file_name, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(results)


def task1(input_path: str, stream_size: int, num_of_asks: int, output_path: str):
    # Initialize Spark
    conf = SparkConf().setAppName("Task 1: Bloom Filter").setMaster("local[*]")
    spark = SparkContext(conf=conf).getOrCreate()
    spark.setLogLevel("ERROR")

    try:
        start_time = time.time()

        # Initialize BlackBox
        blackbox = BlackBox()

        # Apply bloom filter on stream of users
        results = bloom_filter(input_path, blackbox, num_of_asks, stream_size)

        # Write results to output file
        save_output(output_path, results)

        execution_time = time.time() - start_time
        print(f"Duration: {execution_time}\n")

    finally:
        # Stop Spark
        spark.stop()


# if __name__ == "__main__":
#     # Check if correct number of command-line arguments are provided
#     if len(sys.argv) != 5:
#         print("Usage: python task1.py <input_filename> <stream_size> <num_of_asks> <output_filename>")
#         sys.exit(1)

#     # Parse command-line arguments
#     input_path = sys.argv[1]
#     stream_size = int(sys.argv[2])
#     num_of_asks = int(sys.argv[3])
#     output_path = sys.argv[4]

#     # Call task1 function
#     task1(input_path, stream_size, num_of_asks, output_path)

task1(Path.input_csv_file, 100, 30, Path.task1_output)


Duration: 16.860059022903442

time: 29.1 s (started: 2024-04-12 23:50:41 +00:00)


# Task 2

In [27]:
import binascii
import csv
import random
import sys
import time

from blackbox import BlackBox
from pyspark import SparkConf, SparkContext

FILTER_ARRAY_LENGTH = 997
ENCODING = "utf-8"
NUM_HASHES = 50
PRIME_NUMBER = 1e9 + 7


def generate_hash_function_params(max_range, count):
    """Generate random hash function parameters within a specified range."""
    a = random.sample(range(1, max_range), count)  # Random coefficient 'a'
    b = random.sample(range(1, max_range), count)  # Random intercept 'b'
    return list(zip(a, b))


def hash_user(user, params):
    """Hash an item using given hash function parameters.
    Calculate hash value using the formula: ((a * item + b) % PRIME_NUMBER) % num_bins
    """
    user = int(binascii.hexlify(user.encode("utf8")), 16)
    hash_val = ((params[0] * user + params[1]) % PRIME_NUMBER) % FILTER_ARRAY_LENGTH
    return hash_val


def myhashs(user):
    hash_funcs = generate_hash_function_params(FILTER_ARRAY_LENGTH, NUM_HASHES)
    return [hash_user(user, hash_funcs[i]) for i in range(NUM_HASHES)]


def calculate_ground_truth(stream_users, ground_truth):
    for user in stream_users:
        ground_truth.add(user)

    return len(ground_truth)


def calculate_estimation(usr_hash):
    sum_estimate = 0

    for h in range(NUM_HASHES):
        temp = [int(value[h]) for value in usr_hash.values()]

        max_t_zero = 0
        for value in temp:
            tmp_str = bin(value)[2:]
            wo_zero = tmp_str.rstrip('0')
            if max_t_zero < len(tmp_str) - len(wo_zero):
                max_t_zero = len(tmp_str) - len(wo_zero)

        sum_estimate += 2 ** max_t_zero
    return sum_estimate // NUM_HASHES


def flajolet_martin(input_path: str, blackbox: BlackBox, num_of_asks: int, stream_size: int):
    results = []

    # Fetch stream and perform Bloom Filtering for each batch
    for i in range(num_of_asks):
        ground_truth = set()
        usr_hash = dict()

        stream_usrs = blackbox.ask(input_path, stream_size)
        len_ground_truth = calculate_ground_truth(stream_usrs, ground_truth)

        for user in stream_usrs:
            usr_hash[user] = myhashs(user)

        estimate = calculate_estimation(usr_hash)
        results.append([i, len_ground_truth, estimate])

    return results


def save_output(output_file_name, results):
    header = ["Time", "Ground Truth", "Estimation"]
    with open(output_file_name, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(results)


def task2(input_path: str, stream_size: int, num_of_asks: int, output_path: str):
    try:
        start_time = time.time()

        # Initialize BlackBox
        blackbox = BlackBox()

        # Apply bloom filter on stream of users
        results = flajolet_martin(input_path, blackbox, num_of_asks, stream_size)

        # Write results to output file
        save_output(output_path, results)

        execution_time = time.time() - start_time
        print(f"Duration: {execution_time}\n")

    except Exception as e:
        print(e)


# if __name__ == "__main__":
#     # Check if correct number of command-line arguments are provided
#     if len(sys.argv) != 5:
#         print("Usage: python task2.py <input_filename> <stream_size> <num_of_asks> <output_filename>")
#         sys.exit(1)

#     # Parse command-line arguments
#     input_path = sys.argv[1]
#     stream_size = int(sys.argv[2])
#     num_of_asks = int(sys.argv[3])
#     output_path = sys.argv[4]

#     # Call task1 function
#     task2(input_path, stream_size, num_of_asks, output_path)

task2(Path.input_csv_file, 300, 30, Path.task2_output)


Duration: 10.805005550384521

time: 10.8 s (started: 2024-04-13 01:43:14 +00:00)


# Task 3

In [37]:
import csv
import random
import sys
import time

from blackbox import BlackBox

def reservoir_sampling(input_path: str, blackbox: BlackBox, num_of_asks: int, stream_size: int):
    results = []

    users_found = []
    seq_num = 0

    for i in range(num_of_asks):
        stream_usrs = blackbox.ask(input_path, stream_size)

        for usr in stream_usrs:
            seq_num += 1

            # For the first 100 users, add them directly to the reservoir
            if len(users_found) < 100:
                users_found.append(usr)
            elif random.random() < 100 / seq_num:
                replace_index = random.randint(0, 99)
                users_found[replace_index] = usr

            # Output the current stage of the reservoir after every 100 users
            if seq_num % 100 == 0:
                results.append([seq_num] + users_found[::20])

    return results


def save_output(output_file_name, results):
    header = ["seqnum", "0_id", "20_id", "40_id", "60_id", "80_id"]

    with open(output_file_name, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)
        writer.writerows(results)


def task3(input_path: str, stream_size: int, num_of_asks: int, output_path: str):
    random.seed(553)

    try:
        start_time = time.time()

        # Initialize BlackBox
        blackbox = BlackBox()

        # Apply algorithm on stream of users
        results = reservoir_sampling(input_path, blackbox, num_of_asks, stream_size)

        # Write results to output file
        save_output(output_path, results)

        execution_time = time.time() - start_time
        print(f"Duration: {execution_time}\n")

    except Exception as e:
        print(e)


# if __name__ == "__main__":
#     # Check if correct number of command-line arguments are provided
#     if len(sys.argv) != 5:
#         print("Usage: python task3.py <input_filename> <stream_size> <num_of_asks> <output_filename>")
#         sys.exit(1)

#     # Parse command-line arguments
#     input_path = sys.argv[1]
#     stream_size = int(sys.argv[2])
#     num_of_asks = int(sys.argv[3])
#     output_path = sys.argv[4]

#     # Call task1 function
#     task3(input_path, stream_size, num_of_asks, output_path)

task3(Path.input_csv_file, 100, 30, Path.task3_output)


Duration: 8.900791883468628

time: 8.91 s (started: 2024-04-13 02:20:39 +00:00)


# END